In [1]:
import pandas as pd
import numpy as np
import os

import pandas as pd
import matplotlib.pyplot as plt
from sklearn import tree, naive_bayes
import numpy as np
import seaborn as sns
import re
from scipy.stats import chi2_contingency
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import LabelEncoder, StandardScaler
import graphviz
from sklearn.model_selection import cross_val_score, StratifiedKFold, cross_val_score
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, balanced_accuracy_score
import optuna
import string
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
import spacy
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

C:\Users\Bryan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Bryan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Bryan\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Bryan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
train_acl = pd.read_csv("aclimdb/train.csv")
test_acl = pd.read_csv("aclimdb/test.csv")

In [3]:
#Try to extract several variables from text
#simple features
def splitSentenceInPredefinedSeparator(x):
    separators = [',', ' ', '.']
    for separator in separators:
        x = x.replace(separator, ' ')
    return str(x).split()

#Count number of words
def numberOfWords(x):
    #return number of words that has been splitted including with ',', '.', ' '
    return len(splitSentenceInPredefinedSeparator(x))

def uniqueWords(x):
    #return number of unique words that has been splitted including with ',', '.', ' '
    return len(set(splitSentenceInPredefinedSeparator(x)))


train_acl['num-words'] = train_acl['text'].apply(lambda x: numberOfWords(x))
test_acl['num-words'] = test_acl['text'].apply(lambda x: numberOfWords(x))
#Count number of unique words
train_acl['num-unique-words'] = train_acl['text'].apply(lambda x: uniqueWords(x))
test_acl['num-unique-words'] = test_acl['text'].apply(lambda x: uniqueWords(x))
#Count number of punctuations
train_acl['num-punctuations'] = train_acl['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))
test_acl['num-punctuations'] = test_acl['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))
#UniqueWordRatio
train_acl['unique-word-ratio'] = train_acl['num-unique-words'] / train_acl['num-words']
test_acl['unique-word-ratio'] = test_acl['num-unique-words'] / test_acl['num-words']
#PunctuationRatio
train_acl['punctuation-ratio'] = train_acl['num-punctuations'] / train_acl['num-words']
test_acl['punctuation-ratio'] = test_acl['num-punctuations'] / test_acl['num-words']
#Sentence Length
train_acl['sentence-length'] = train_acl['text'].apply(lambda x: len(x.split('.')))
test_acl['sentence-length'] = test_acl['text'].apply(lambda x: len(x.split('.')))
#Number of Words beginning with capital letters (Other than first word in a sentence after period)
train_acl['num-words-beginning-capital'] = train_acl['text'].apply(lambda x: len([w for w in x.split() if w[0].isupper() and w[0] not in ['.', '"', "'"]]))
test_acl['num-words-beginning-capital'] = test_acl['text'].apply(lambda x: len([w for w in x.split() if w[0].isupper() and w[0] not in ['.', '"', "'"]]))
# #Count Number of Nouns, and Nouns Ratio
# print("beginning noun extraction")
# train_acl['num-nouns'] = train_acl['text'].apply(lambda x: len([w for w in word_tokenize(x) if pos_tag([w])[0][1] in ['NN', 'NNP']]))
# test_acl['num-nouns'] = test_acl['text'].apply(lambda x: len([w for w in word_tokenize(x) if pos_tag([w])[0][1] in ['NN', 'NNP']]))
# train_acl['nouns-ratio'] = train_acl['num-nouns'] / train_acl['num-words']
# test_acl['nouns-ratio'] = test_acl['num-nouns'] / test_acl['num-words']
# #Count Number of Verbs, and Verbs Ratio
# print("beginning verb extraction")
# train_acl['num-verbs'] = train_acl['text'].apply(lambda x: len([w for w in word_tokenize(x) if pos_tag([w])[0][1] in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']]))
# test_acl['num-verbs'] = test_acl['text'].apply(lambda x: len([w for w in word_tokenize(x) if pos_tag([w])[0][1] in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']]))
# train_acl['verbs-ratio'] = train_acl['num-verbs'] / train_acl['num-words']
# test_acl['verbs-ratio'] = test_acl['num-verbs'] / test_acl['num-words']
# #Count Number of Adjectives, and Adjectives Ratio
# print("beginning adjective extraction")
# train_acl['num-adjectives'] = train_acl['text'].apply(lambda x: len([w for w in word_tokenize(x) if pos_tag([w])[0][1] in ['JJ', 'JJR', 'JJS']]))
# test_acl['num-adjectives'] = test_acl['text'].apply(lambda x: len([w for w in word_tokenize(x) if pos_tag([w])[0][1] in ['JJ', 'JJR', 'JJS']]))
# train_acl['adjectives-ratio'] = train_acl['num-adjectives'] / train_acl['num-words']
# test_acl['adjectives-ratio'] = test_acl['num-adjectives'] / test_acl['num-words']
# #Average Word Length
# train_acl['average-word-length'] = train_acl['text'].apply(lambda x: np.mean([len(w) for w in splitSentenceInPredefinedSeparator(x)]))
# test_acl['average-word-length'] = test_acl['text'].apply(lambda x: np.mean([len(w) for w in splitSentenceInPredefinedSeparator(x)]))
# #stopwords count
# train_acl['stopwords-count'] = train_acl['text'].apply(lambda x: len([w for w in splitSentenceInPredefinedSeparator(x) if w.lower() in nltk.corpus.stopwords.words('english')]))
# test_acl['stopwords-count'] = test_acl['text'].apply(lambda x: len([w for w in splitSentenceInPredefinedSeparator(x) if w.lower() in nltk.corpus.stopwords.words('english')]))
# train_acl['Stopwords-ratio'] = train_acl['stopwords-count'] / test_acl['num-words']
# test_acl['Stopwords-ratio'] = test_acl['stopwords-count'] / test_acl['num-words']

In [4]:
#Adjective extraction
print("beginning Adjective extraction")
train_num_adj = []
test_num_adj = []
for i in range(len(train_acl)):
    if i % 100 == 0:
        print(i)
    train_num_adj.append(len([w for w in word_tokenize(train_acl['text'][i]) if pos_tag([w])[0][1] in ['JJ', 'JJR', 'JJS']]))
    test_num_adj.append(len([w for w in word_tokenize(test_acl['text'][i]) if pos_tag([w])[0][1] in ['JJ', 'JJR', 'JJS']]))
train_acl['num-adjs'] = train_num_adj
test_acl['num-adjs'] = test_num_adj

beginning Adjective extraction
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
1

In [5]:
train_acl['adjs-ratio'] = train_acl['num-adjs'] / train_acl['num-words']
test_acl['adjs-ratio'] = test_acl['num-adjs'] / test_acl['num-words']

In [6]:
train_acl.to_csv("aclimdb/train_heuristic_adjs.csv", index=False)
test_acl.to_csv("aclimdb/test_heuristic_adjs.csv", index=False)